In [97]:
!pip install mysql-connector-python langchain langchain_openai openai langchain_experimental sqlalchemy pinecone-client -q

You should consider upgrading via the '/Users/danielhernandez/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [98]:
!pip install pandas tabulate

You should consider upgrading via the '/Users/danielhernandez/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [ ]:
#Gives ip address
!hostname -I

#gives ip address
!curl ipecho.net/plain

In [99]:
import sqlalchemy
import pandas as pd

database_username = "username"
database_password = "password"
database_ip = "hostname"
database_name = "db-name"
database_port = "db-port"

database_connection = sqlalchemy.create_engine(
    "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}".format(
        database_username, database_password, database_ip, database_port, database_name
    )
)

comments = pd.read_sql_query("""SELECT * FROM all_star_comment""", con=database_connection)

comments['date'] = pd.to_datetime(comments['date'], unit='ms')

data = comments.to_json(orient='records', force_ascii=False, date_format='iso')

In [100]:
from getpass import getpass
import os

PINECONE_API_KEY = ''
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

OPENAI_API_KEY = ''
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
from pinecone import Pinecone

pc = Pinecone()

index_name = 'langchain'
index = pc.Index(index_name)

In [101]:
# load core modules
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
# load agents and tools modules
import pandas as pd
from io import StringIO
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

import pandas as pd
from langchain_openai import OpenAI

agent_kwargs = {}

agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY),
    comments,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_kwargs=agent_kwargs
)

query = 'Como puedo mejorar mi negocio'

template = f'''
Limítate a contestar preguntas solamente relevantes a los datos entregados.

Esta pregunta no está relacionada a Pandas.

Eres un experto analista de customer feedback. Especializado en encontrar patrones y tendencias en el feedback que los negocios reciben por medio de los comentarios de los clientes en encuestas de satisfacción u otros canales de comunicación como redes sociales y mensajeria de texto.

Los datos que normalmente maneja un experto en customer feedback son de tipo texto para hacer análisis cualitativo a nivel especifico, revisando lo que cada cliente esta diciendo, y a nivel general identificando patrones y tendencias agregadas de los comentarios individuales de los clientes.

El objetivo de un expert customer feedback analyst es encontrar oportunidades de mejora que van mas allá de solo identificar puntos importantes de una buena atención al cliente sino hasta la capacidad de identificar oportunidades de mejora en la experiencia completa de los clientes en el negocio donde labora. Ya que los comentarios de los clientes revelan mucho más que un buen servicio al cliente.

Considera los datos de pandas cargados y responde de manera inteligente y no génerica a las entradas del usuario. 

No uses la palabra dataframe nunca.

Todas las respuestas deben estar resumidas. Solo da respuestas largar cuando sea realmente necesario.

\nUSER INPUT: {query}
\nAI RESPONSE HERE:
'''

output = agent.invoke(template)


In [ ]:
output['output']

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import pandas as pd

memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"output": "whats up"})

memory.load_memory_variables({})

# Function to check if the query is relevant to the comments dataframe
def is_query_relevant(query, dataframe):
    # Example of a simple relevance check
    keywords = [
        'servicio', 'producto', 'calidad', 'entregar', 'cliente', 'negocio', 'experiencia', 'consumidor', 
        'comentario', 'comentarios', 'feedback', 'retroalimentación', 'datos'
    ]

    return any(keyword in query.lower() for keyword in keywords)

# Custom Agent Class to handle out-of-scope queries
class CustomDataFrameAgent:
    def __init__(self, base_agent, dataframe):
        self.base_agent = base_agent
        self.dataframe = dataframe

    def invoke(self, query):
        if is_query_relevant(query, self.dataframe):
            return self.base_agent.invoke(query)
        else:
            return "Soy un experto analista de customer feedback. Por favor intenta ser un poco más específico."

# Initialize your agent
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY),
    comments,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

# Wrap your agent in the custom class
custom_agent = CustomDataFrameAgent(agent, comments)

template = f'''
Eres un experto analista de customer feedback. Especializado en encontrar patrones y tendencias en el feedback.

El objetivo de un expert customer feedback analyst es encontrar oportunidades de mejora que van mas allá de solo identificar puntos importantes de una buena atención al cliente sino hasta la capacidad de identificar oportunidades de mejora en la experiencia completa de los clientes en el negocio donde labora. Ya que los comentarios de los clientes revelan mucho más que un buen servicio al cliente.

No puedes responder nada relacionado a Pandas o Python.

No uses la palabra "Dataframe" o "df".

La respuesta deben estar resumida.

No menciones nombres de columnas.

No digas nada como esto: "Para obtener información sobre los clientes en el dataframe `df`, puedes obtener los siguientes datos:"

\nUSER INPUT: {query}
\nAI RESPONSE HERE:
'''

# Example query and output
query = 'Que puedo ofrecer a mis clientes?'
# output = custom_agent.invoke(query)
# print(output)

In [ ]:
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

# Setup database
db = SQLDatabase.from_uri(
    "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}".format(
        database_username, database_password, database_ip, database_port, database_name
    )
)

# setup llm
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# Create db chain
QUERY = """
Eres un experto analista de retroalimentación de clientes, especializado en identificar patrones y tendencias en la retroalimentación.

El objetivo de un experto analista de retroalimentación de clientes es encontrar oportunidades de mejora que vayan más allá de simplemente identificar puntos clave de buen servicio al cliente, sino también identificar oportunidades de mejora en la experiencia general del cliente dentro del negocio donde trabajan. Ya que los comentarios de los clientes revelan mucho más que solo un buen servicio al cliente.

Dada la pregunta, mire la tabla 'comments' SI ES NECESARIO y dame una respuesta que cumpla con tu objetivo como experto analista de retroalimentación.

Pregunta: {question}
Respuesta: Respuesta final aquí
"""

# Setup the database chain
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)


def get_prompt():
    print("Type 'exit' to quit")

    while True:
        prompt = input("Enter a prompt: ")

        if prompt.lower() == 'exit':
            print('Exiting...')
            break
        else:
            try:
                question = QUERY.format(question=prompt)
                print(db_chain.run(question))
            except Exception as e:
                print(e)

# get_prompt()


In [111]:
import json
data = comments.to_json(orient='records', force_ascii=False, date_format='iso')
data = json.loads(data)
print(data[0])

{'id': 1, 'company_id': 1, 'date': '2021-01-31T08:29:34.000', 'stars': 5, 'comment': 'El servicio en este hotel es excepcional, el personal siempre está atento y dispuesto a ayudar.', 'name': 'Daniel', 'email': 'daniel@gmail.com', 'phone': '342424234'}


In [103]:
converted_data = {}
for d in data:
    converted_data.update({d['id']: d['comment']})

converted_data

{1: 'El servicio en este hotel es excepcional, el personal siempre está atento y dispuesto a ayudar.',
 2: 'Las instalaciones son impecables, todo está muy limpio y bien mantenido',
 3: 'La comida en el restaurante del hotel es deliciosa, con una amplia variedad de opciones para todos los gustos.',
 4: 'La ubicación es perfecta, cerca de las principales atracciones turísticas y con fácil acceso al transporte público.',
 5: 'Las habitaciones son cómodas y espaciosas, ideales para relajarse después de un día de turismo.',
 6: 'El Wi-Fi funciona perfectamente en todo el hotel, lo cual es genial para aquellos que necesitan estar conectados.',
 7: 'El desayuno buffet es abundante y variado, con opciones frescas y de alta calidad.',
 8: 'El ambiente del hotel es muy acogedor y tranquilo, ideal para disfrutar de una estancia relajante.',
 9: 'El spa y las áreas de bienestar son excelentes, perfectas para consentirse y relajarse.',
 10: 'El hotel ofrece una gran cantidad de servicios adicional

In [112]:
context = comments.to_string(index=False)
context

' id  company_id                date  stars                                                                                                                                                                           comment   name            email     phone\n  1           1 2021-01-31 08:29:34      5                                                                                   El servicio en este hotel es excepcional, el personal siempre está atento y dispuesto a ayudar. Daniel daniel@gmail.com 342424234\n  2           1 2021-01-31 08:29:34      5                                                                                                           Las instalaciones son impecables, todo está muy limpio y bien mantenido Daniel daniel@gmail.com 342424234\n  3           1 2021-01-31 08:29:34      4                                                                    La comida en el restaurante del hotel es deliciosa, con una amplia variedad de opciones para todos los gustos. Daniel dan

In [139]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_openai_tools_agent

data_description = comments.to_string(index=False)


query = 'Cuando fue la segunda guerra mundial?'

from langchain_experimental.utilities import PythonREPL
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True,
    agent_executor_kwargs={"handle_parsing_errors": True}
)

prompt = f"""

Eres un experto analista de retroalimentación de clientes, especializado en identificar patrones y tendencias en la retroalimentación.

El objetivo de un experto analista de retroalimentación de clientes es encontrar oportunidades de mejora que vayan más allá de simplemente identificar puntos clave de buen servicio al cliente, sino también identificar oportunidades de mejora en la experiencia general del cliente dentro del negocio donde trabajan. Ya que los comentarios de los clientes revelan mucho más que solo un buen servicio al cliente.

Si no sabes la respuesta a una pregunta, simplete response, no sé.

Responde la siguiente pregunta: {query}, analizando los siguientes datos {data_description} 

"""
response = agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I don't know
Observation: Invalid Fo

In [140]:
print(response['output'])

Agent stopped due to iteration limit or time limit.
